In [3]:

from google.colab import drive

drive.mount('/content/gdrive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive')





In [0]:
import helpers_strings as hs

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pylab as plt
from sklearn.metrics import f1_score

from nltk import word_tokenize, sent_tokenize
import nltk
# nltk.download('punkt')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# from gensim.models.word2vec import Word2Vec
# from gensim.models import KeyedVectors

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, GlobalMaxPooling1D, Bidirectional, GRU, CuDNNGRU, TimeDistributed, CuDNNLSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier


Using TensorFlow backend.


In [0]:
from keras.layers import Input
from keras.models import Model

In [0]:
datos = pd.read_csv('/content/gdrive/My Drive/train.csv')
X = datos['text']
y = datos['target']

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=2, random_state=1)

X_train = pd.Series([hs.full_clean_text(t) for t in X_train])
X_val = pd.Series([hs.full_clean_text(t) for t in X_val])

vocab_size = 1000+1 # Adding 1 because of reserved 0 index
tokenizer = Tokenizer(num_words= vocab_size)
tokenizer.fit_on_texts(X)
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

print(X_train[2])
print(X_val[1])

[6, 15, 258, 512, 61, 42, 36]
[4, 428, 49, 222, 237, 6, 4, 8]


In [9]:
maxlen=200
X_train_padded = pad_sequences(X_train,padding ='post',  maxlen = maxlen)
X_val_padded   = pad_sequences(X_val, padding = 'post', maxlen = maxlen)
X_train_padded.shape

(7611, 200)

In [156]:
# Sanity Check
sanity_x = X_train_padded[0:20]
print(sanity_x)
sanity_y = y_train[0:20]
print(sanity_y)

[[203  45 575 ...   0   0   0]
 [ 10 670 119 ...   0   0   0]
 [296  15 255 ...   0   0   0]
 ...
 [832 242 315 ...   0   0   0]
 [681   8 134 ...   0   0   0]
 [ 47  37 816 ...   0   0   0]]
1163    1
4989    0
6913    1
6711    1
2736    1
7290    0
2756    0
1723    0
6984    0
1624    1
7258    0
181     0
4088    1
5804    1
6486    0
1735    1
5661    0
7601    1
6365    1
3144    0
Name: target, dtype: int64


In [0]:
# si pongo el nuevo objeto optimizer en el compile
# agarre learn rate del param grid o es solo para parametros de .fit() ?

def run_bigru(batch_size=100, epochs=10, validation_data=(X_val_padded, y_val), sanity = False, learn_rate = 0.001):
  optimizer = Adam(lr= learn_rate)
  embedding_dim = 200
  model_bigru = Sequential()
  model_bigru.add(Embedding(vocab_size, embedding_dim))
  model_bigru.add(Bidirectional(CuDNNGRU(128, return_sequences = True)))
  model_bigru.add(Bidirectional(CuDNNGRU(128)))
  model_bigru.add(Dense(128, activation='relu'))
  model_bigru.add(Dense(128, activation='relu'))
  model_bigru.add(Dense(1, activation='sigmoid'))
  model_bigru.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])
  # if sanity:
  #   model_bigru.fit(x,y, batch_size = 1, epochs = 5)
  # else:
  #   model_bigru.fit(x,y, batch_size = batch_size, epochs = epochs, validation_data= validation_data)
  return model_bigru


In [179]:
# Sanity Check
run_bigru(sanity_x, sanity_y, sanity = True)

Epoch 1/5
20/20 [==============================] - 7s 350ms/step - loss: 0.7067 - acc: 0.3500
Epoch 2/5
20/20 [==============================] - 0s 25ms/step - loss: 0.6924 - acc: 0.5000
Epoch 3/5
20/20 [==============================] - 0s 24ms/step - loss: 0.4728 - acc: 0.8500
Epoch 4/5
20/20 [==============================] - 0s 24ms/step - loss: 0.0015 - acc: 1.0000
Epoch 5/5
20/20 [==============================] - 0s 25ms/step - loss: 1.7091e-06 - acc: 1.0000


In [11]:
keras_bigru = KerasClassifier(build_fn=run_bigru) 

learn_rate = [0.0001, 0.001, 0.01]
batch_size = [8,32,64,128]
epochs = [10,20]

param_grid = dict(learn_rate=learn_rate, 
                  batch_size=batch_size,
                  epochs = epochs)

grid = GridSearchCV(estimator=keras_bigru, 
                    param_grid=param_grid,
                    cv = 5)

grid_result = grid.fit(X_train_padded, y_train)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





6088/6088 [==============================] - 43s 7ms/step - loss: 0.5249 - acc: 0.7470
Epoch 2/10
6088/6088 [==============================] - 28s 5ms/step - loss: 0.4375 - acc: 0.8070
Epoch 3/10
6088/6088 [==============================] - 28s 5ms/step - loss: 0.4050 - acc: 0.8221
Epoch 4/10
6088/6088 [==============================] - 28s 5ms/step - loss: 0.3670 - acc: 0.8436
Epoch 5/10
6088/6088 [==============================] - 28s 5ms/step - loss: 0.3330 - acc: 0.8622
Epoch 6/10
6088/6088 [==============================] - 29s 5ms/step - loss: 0.2980 - acc: 0.8743
Epoch 7/10
6088/6088 [==============================] - 28s 5ms/step - loss: 0.2584 - acc: 0.8945
Epoch 8/10
6088/6088 [==============================] - 28s 5ms/step - loss: 0.2346 - acc: 0.9033
Epoch 9/10
6088/6088 [==============================] - 28s 5ms/step - loss: 0.2033 - acc: 0.9136
Epoch 10/10

NameError: ignored

In [0]:
results = pd.DataFrame(grid_result.cv_results_)

In [0]:
results.to_csv('bigru_cv.csv')
!cp bigru_cv.csv "/content/gdrive/My Drive/"

In [181]:
# Modelo
run_bigru(X_train_padded, y_train, batch_size = 32, epochs= 8, validation_data=(X_val_padded, y_val))

Train on 6090 samples, validate on 1523 samples
Epoch 1/8
6090/6090 [==============================] - 18s 3ms/step - loss: 0.5336 - acc: 0.7337 - val_loss: 0.4804 - val_acc: 0.7827
Epoch 2/8
6090/6090 [==============================] - 11s 2ms/step - loss: 0.4335 - acc: 0.8067 - val_loss: 0.4765 - val_acc: 0.7938
Epoch 3/8
6090/6090 [==============================] - 11s 2ms/step - loss: 0.3955 - acc: 0.8259 - val_loss: 0.4704 - val_acc: 0.7853
Epoch 4/8
6090/6090 [==============================] - 11s 2ms/step - loss: 0.3681 - acc: 0.8401 - val_loss: 0.5131 - val_acc: 0.7873
Epoch 5/8
6090/6090 [==============================] - 11s 2ms/step - loss: 0.3437 - acc: 0.8586 - val_loss: 0.5058 - val_acc: 0.7768
Epoch 6/8
6090/6090 [==============================] - 11s 2ms/step - loss: 0.3133 - acc: 0.8683 - val_loss: 0.5690 - val_acc: 0.7702
Epoch 7/8
6090/6090 [==============================] - 11s 2ms/step - loss: 0.2806 - acc: 0.8833 - val_loss: 0.5842 - val_acc: 0.7754
Epoch 8/8
6090

In [0]:
learn_rate = 0.001
optimizer = Adam(lr= learn_rate)

def run_lstm(x,y, batch_size=100, epochs=10, validation_data=(X_val_padded, y_val), sanity = False):  
  embedding_dim = 200
  model = Sequential()
  model.add(Embedding(vocab_size, embedding_dim))
  model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
  model.add(Bidirectional(CuDNNLSTM(128)))
  model.add(Dense(128, activation="relu"))
  model.add(Dense(128, activation="relu"))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer= optimizer, metrics=['accuracy'])
  if sanity:
    model.fit(x,y, batch_size = 1, epochs = 5)
  else:
    model.fit(x,y, batch_size = batch_size, epochs = epochs, validation_data= validation_data)
  return model

In [220]:
# Sanity Check
run_lstm(sanity_x, sanity_y, sanity = True)

Epoch 1/5
20/20 [==============================] - 10s 500ms/step - loss: 0.7014 - acc: 0.3000
Epoch 2/5
20/20 [==============================] - 1s 30ms/step - loss: 0.6946 - acc: 0.5000
Epoch 3/5
20/20 [==============================] - 1s 31ms/step - loss: 0.6511 - acc: 0.8000
Epoch 4/5
20/20 [==============================] - 1s 34ms/step - loss: 0.1036 - acc: 1.0000
Epoch 5/5
20/20 [==============================] - 1s 34ms/step - loss: 5.5214e-04 - acc: 1.0000


In [223]:
# Modelo
run_lstm(X_train_padded, y_train, batch_size = 32, epochs= 8, validation_data=(X_val_padded, y_val))

Train on 6090 samples, validate on 1523 samples
Epoch 1/8
6090/6090 [==============================] - 25s 4ms/step - loss: 0.5375 - acc: 0.7310 - val_loss: 0.4731 - val_acc: 0.7919
Epoch 2/8
6090/6090 [==============================] - 13s 2ms/step - loss: 0.4333 - acc: 0.8064 - val_loss: 0.4659 - val_acc: 0.7945
Epoch 3/8
6090/6090 [==============================] - 13s 2ms/step - loss: 0.4058 - acc: 0.8251 - val_loss: 0.4617 - val_acc: 0.7978
Epoch 4/8
6090/6090 [==============================] - 13s 2ms/step - loss: 0.3789 - acc: 0.8350 - val_loss: 0.4865 - val_acc: 0.7768
Epoch 5/8
6090/6090 [==============================] - 13s 2ms/step - loss: 0.3472 - acc: 0.8525 - val_loss: 0.5391 - val_acc: 0.7886
Epoch 6/8
6090/6090 [==============================] - 13s 2ms/step - loss: 0.3240 - acc: 0.8624 - val_loss: 0.5367 - val_acc: 0.7682
Epoch 7/8
6090/6090 [==============================] - 13s 2ms/step - loss: 0.2972 - acc: 0.8695 - val_loss: 0.6669 - val_acc: 0.7695
Epoch 8/8
6090